In [12]:
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense
from keras.models import Model, Sequential, load_model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.backend import mean, square

from spektral.datasets import qm9
from spektral.layers import EdgeConditionedConv, GlobalAttentionPool
from spektral.utils import label_to_one_hot

from os import path
import itertools

Using TensorFlow backend.


In [2]:
A, X, E, y = qm9.load_data(return_type='numpy',
                          nf_keys='atomic_num',
                          ef_keys='type',
                          self_loops=True,
                          amount=10000)
uniq_X = np.unique(X)
X = label_to_one_hot(X, uniq_X)

Loading QM9 dataset.
Reading SDF


100%|██████████| 10000/10000 [00:04<00:00, 2125.79it/s]


In [3]:
tasks = list(y.columns)[1:]
y_list = []
for task in tasks:
    y_list.append(y[[task]].values)
for i in range(len(y_list)):
    y_list[i] = StandardScaler().fit_transform(y_list[i]).reshape(-1, y_list[0].shape[-1])

In [4]:
print(tasks)

['A', 'B', 'C', 'mu', 'alpha', 'homo', 'lumo', 'gap', 'r2', 'zpve', 'u0', 'u298', 'h298', 'g298', 'cv', 'u0_atom', 'u298_atom', 'h298_atom', 'g298_atom']


In [5]:
N = X.shape[-2]
F = X.shape[-1] 
S = E.shape[-1]
n_out = y_list[0].shape[-1]
learning_rate = 1e-3
epochs = 3
batch_size = 64
es_patience = 5

In [6]:
optimizer = Adam(lr=learning_rate)
loss = 'mse'

In [7]:
A_train, A_test, \
X_train, X_test, \
E_train, E_test, \
*y_train_test_list = train_test_split(A, X, E, *y_list, test_size = 0.1)

y_train_list = y_train_test_list[::2]
y_test_list = y_train_test_list[1::2]

In [8]:
X_in = Input(shape=(N, F))
A_in = Input(shape=(N, N))
E_in = Input(shape=(N, N, S))

In [9]:
def create_single_task_model(X_in, A_in, E_in):
    gc1 = EdgeConditionedConv(64, activation='relu')([X_in, A_in, E_in])
    gc2 = EdgeConditionedConv(128, activation='relu')([gc1, A_in, E_in])
    pool = GlobalAttentionPool(256)(gc2)
    dense = Dense(256, activation='relu')(pool)
    output = Dense(n_out)(dense)
    return Model(inputs=[X_in, A_in, E_in], outputs=output)

In [10]:
def generate_filename(task):
    return path.join('single_task_trained_models', task + '.h5')

In [11]:
# for i in range(-2, 0):
#     print('learning', tasks[i])
#     model = create_single_task_model(X_in, A_in, E_in)
#     model.compile(optimizer=optimizer, loss=loss)
#     es_callback = EarlyStopping(monitor='val_loss', patience=es_patience)
#     model.fit([X_train, A_train, E_train],
#              y_train_list[i],
#              batch_size=batch_size,
#              validation_split=0.1,
#              epochs=epochs,
#              callbacks=[es_callback])
#     model.save_weights(generate_filename(tasks[i]))

In [12]:
def load_and_compile_model(task):
    model = create_single_task_model(X_in, A_in, E_in)
    model.load_weights(generate_filename(task))
    model.compile(optimizer=optimizer, loss=loss)

In [13]:
def calculate_transfer_coefficient_from_file(task_learned, task_transferred, y_transferred_test):
    model_learned = create_single_task_model(X_in, A_in, E_in)
    model_transferred = create_single_task_model(X_in, A_in, E_in)
    model_learned.load_weights(generate_filename(task_learned))
    model_transferred.load_weights(generate_filename(task_transferred))
    
    layers_learned = model_learned.get_weights()
    layers_transferred = model_transferred.get_weights()    
    layers_transferred = layers_learned[:10] + layers_transferred[10:]
    model_transferred.set_weights(layers_transferred)
    model_transferred.compile(optimizer=optimizer, loss=loss)
    
    eval_results = model_transferred.evaluate([X_test, A_test, E_test],
                                              y_transferred_test,
                                              batch_size=batch_size)
    return eval_results

In [14]:
def load_and_test_model(task_learned, y_test):
#     model_learned = load_model(path.join('single_task_trained_models', task_learned+'.h5'), 
#                                custom_objects={'EdgeConditionedConv':EdgeConditionedConv,
#                                               'GlobalAttentionPool':GlobalAttentionPool})
    model = create_single_task_model(X_in, A_in, E_in)
    model.load_weights(path.join('single_task_trained_models', task_learned + '.h5'))
    model.compile(optimizer=optimizer, loss=loss)
    eval_results = model.evaluate([X_test, A_test, E_test], y_test, batch_size=batch_size)
    return eval_results

In [15]:
print(load_and_test_model('A', y_test_list[0]))
print(calculate_transfer_coefficient_from_file('A', 'B', y_test_list[1]))





1000/1000 [==============================] - 3s 3ms/step
0.0001221810671268031
1000/1000 [==============================] - 2s 2ms/step
0.13086552423238754


In [16]:
print(tasks)
transfer_coefficient_dict = dict()
for i, j in itertools.permutations(range(len(tasks)), 2):
    task_learned = tasks[i]
    task_transferred = tasks[j]
    print(task_learned, task_transferred)
    transfer_coefficient = calculate_transfer_coefficient_from_file(task_learned, task_transferred, y_test_list[j])
    print(transfer_coefficient)
    
    if task_learned not in transfer_coefficient_dict.keys():
        transfer_coefficient_dict[task_learned] = {task_transferred: transfer_coefficient}
    else:
        transfer_coefficient_dict[task_learned][task_transferred] = transfer_coefficient

['A', 'B', 'C', 'mu', 'alpha', 'homo', 'lumo', 'gap', 'r2', 'zpve', 'u0', 'u298', 'h298', 'g298', 'cv', 'u0_atom', 'u298_atom', 'h298_atom', 'g298_atom']
A B
1000/1000 [==============================] - 2s 2ms/step
0.13086552423238754
A C
1000/1000 [==============================] - 2s 2ms/step
0.04075440979003906
A mu
1000/1000 [==============================] - 2s 2ms/step
0.9960114765167236
A alpha
1000/1000 [==============================] - 2s 2ms/step
0.9690558757781983
A homo
1000/1000 [==============================] - 2s 2ms/step
1.0085037326812745
A lumo
1000/1000 [==============================] - 2s 2ms/step
1.0072671127319337
A gap
1000/1000 [==============================] - 2s 2ms/step
0.9788152894973755
A r2
1000/1000 [==============================] - 2s 2ms/step
0.9844315662384033
A zpve
1000/1000 [==============================] - 2s 2ms/step
0.985578049659729
A u0
1000/1000 [==============================] - 2s 2ms/step
0.9353220615386962
A u298
1000/1000 [=========

1000/1000 [==============================] - 7s 7ms/step
1.2171734657287598
u0 g298
1000/1000 [==============================] - 7s 7ms/step
0.9670496797561645
u0 cv
1000/1000 [==============================] - 7s 7ms/step
1.0171870241165162
u0 u0_atom
1000/1000 [==============================] - 7s 7ms/step
0.951528730392456
u0 u298_atom
1000/1000 [==============================] - 7s 7ms/step
1.0407440376281738
u0 h298_atom
1000/1000 [==============================] - 7s 7ms/step
1.1304818649291992
u0 g298_atom
1000/1000 [==============================] - 7s 7ms/step
1.0581774253845215
u298 A
1000/1000 [==============================] - 7s 7ms/step
0.0016547212060540914
u298 B
1000/1000 [==============================] - 7s 7ms/step
0.460149254322052
u298 C
1000/1000 [==============================] - 7s 7ms/step
0.03457475543022156
u298 mu
1000/1000 [==============================] - 7s 7ms/step
1.0369175081253053
u298 alpha
1000/1000 [==============================] - 8s 8ms/step
0

In [18]:
file = open('cross_task_transfer_coefficients.txt', 'w')
for key1, value in transfer_coefficient_dict.items():
    for key2, coef in value.items():
        print(key1, key2, coef)
        print(key1, key2, coef, file=file)

A B 0.13086552423238754
A C 0.04075440979003906
A mu 0.9960114765167236
A alpha 0.9690558757781983
A homo 1.0085037326812745
A lumo 1.0072671127319337
A gap 0.9788152894973755
A r2 0.9844315662384033
A zpve 0.985578049659729
A u0 0.9353220615386962
A u298 0.9660965385437011
A h298 0.9426877708435059
A g298 0.9355496406555176
A cv 0.9767080888748169
A u0_atom 0.9614292507171631
A u298_atom 0.9642000150680542
A h298_atom 0.9620214529037475
A g298_atom 0.9760090866088867
B A 0.029335276633501054
B C 1.4973543186187743
B mu 6.117791648864746
B alpha 0.8922842006683349
B homo 1.5726523342132568
B lumo 1.0564279127120972
B gap 1.2640917606353759
B r2 1.3206057624816894
B zpve 1.3429386072158813
B u0 2.2699512557983397
B u298 1.0004086561203003
B h298 1.127336908340454
B g298 1.0460615282058716
B cv 1.1535516033172608
B u0_atom 1.169869665145874
B u298_atom 1.0526442079544067
B h298_atom 1.04954691696167
B g298_atom 3.632545967102051
C A 0.08952055609226227
C B 3.8643195152282717
C mu 1.30467

In [21]:
def analyze_transfer_coefficients():
    with open('cross_task_transfer_coefficients.txt', 'r') as file:
        lines = file.readlines()
        transfer_coefficient_dict = dict()
        for line in lines:
            task_learned, task_transferred, transfer_coefficient = line.strip().split()
            if task_learned not in transfer_coefficient_dict.keys():
                transfer_coefficient_dict[task_learned] = {task_transferred: float(transfer_coefficient)}
            else:
                transfer_coefficient_dict[task_learned][task_transferred] = float(transfer_coefficient)
        return transfer_coefficient_dict

In [22]:
transfer_coefficient = analyze_transfer_coefficients()
coef_list = list()
for i in transfer_coefficient.keys():
    for j in transfer_coefficient[i].keys():
        coef_list.append(transfer_coefficient[i][j])
coef_list = np.asarray(coef_list)
coef_list = np.sort(coef_list)
print(coef_list)

[1.50642847e-04 5.49448672e-04 5.61551695e-04 7.35626511e-04
 1.24972527e-03 1.65472121e-03 2.26056431e-03 2.35274713e-03
 2.54770142e-03 3.20199721e-03 3.50989799e-03 4.45808730e-03
 4.47157137e-03 7.39110517e-03 8.11139455e-03 8.33374753e-03
 2.93352766e-02 3.18984660e-02 3.45747554e-02 4.07544098e-02
 4.25449280e-02 4.79848567e-02 5.02191802e-02 5.98246386e-02
 7.28917348e-02 7.60061885e-02 7.96665972e-02 8.95205561e-02
 1.07230070e-01 1.09092377e-01 1.17242838e-01 1.18422849e-01
 1.20635336e-01 1.21971675e-01 1.30865524e-01 1.41346092e-01
 1.44556050e-01 1.47789176e-01 1.69487599e-01 1.97905662e-01
 2.09468838e-01 2.11503145e-01 2.22263633e-01 2.47053597e-01
 2.59064261e-01 3.02745215e-01 3.30580804e-01 4.60149254e-01
 4.87156391e-01 5.56571133e-01 7.36447355e-01 7.98467746e-01
 8.62830132e-01 8.64207458e-01 8.76264582e-01 8.78760681e-01
 8.82788387e-01 8.83124572e-01 8.83621778e-01 8.84457932e-01
 8.89248106e-01 8.92022728e-01 8.92284201e-01 8.92552869e-01
 8.93602315e-01 8.936726

In [39]:
for i, j in itertools.combinations(transfer_coefficient.keys(), 2):
    if transfer_coefficient[i][j] < 0.9 and transfer_coefficient[j][i] < 0.9:
        print(i, 
              j, 
              round(transfer_coefficient[i][j], 3), 
              round(transfer_coefficient[j][i], 3))

A B 0.131 0.029
A C 0.041 0.09
B alpha 0.892 0.032
zpve u298_atom 0.736 0.864
u298 g298_atom 0.894 0.897
